In [ ]:
# import libraries
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from shapely.geometry import Point

In [ ]:
# import functions
from functions.routes_api_functions import (
    calculateRoute,
    createDataFrameFromResults,
    fetch_all_routes,
    parseDataframeToLocations,
)

## Calculate routes at 3AM on 300 OD pairs

In [ ]:
# import data
od_300 = pd.read_csv("../Data/route_result_uber_hod3am_300_all_original.csv")

In [ ]:
od_300

In [ ]:
origins = od_300[["oid", "oy", "ox"]]
destinations = od_300[["did", "dy", "dx"]]

In [ ]:
# convert origins, destinations to geodataframe
origins = gpd.GeoDataFrame(
    origins,
    geometry=[Point(xy) for xy in zip(origins.ox, origins.oy, strict=False)],
)
destinations = gpd.GeoDataFrame(
    destinations,
    geometry=[Point(xy) for xy in zip(destinations.dx, destinations.dy, strict=False)],
)

In [ ]:
# apply parsedataframetoLocations
originsLoc = parseDataframeToLocations(origins)
destinationsLoc = parseDataframeToLocations(destinations)

In [ ]:
# api key
key = "Jessica API Key"

In [ ]:
# calculate routes
# Nov 29 (Wednesday) 3AM
departure_times = [(2023, 11, 29, hour, 0, 0) for hour in od_300["hour_of_day"]]
# use departure_times list as input
results = fetch_all_routes(
    apikey=key,
    origins=originsLoc,
    destinations=destinationsLoc,
    traffic="TRAFFIC_AWARE",
    departure_times=departure_times,
)

In [ ]:
# test of calculateRoute function
result = calculateRoute(
    apikey=key,
    orig=originsLoc[17],
    dest=destinationsLoc[17],
    traffic="TRAFFIC_AWARE",
    departure_time=(2023, 11, 29, od_300["hour_of_day"][17], 0, 0),
)
result

In [ ]:
df = createDataFrameFromResults(results)
df

In [ ]:
# check location of O-D
# latitude and longitude for the two points
o_lat, o_lon = float(originsLoc[17].lat), float(originsLoc[17].lon)
d_lat, d_lon = float(destinationsLoc[17].lat), float(destinationsLoc[17].lon)

# create a map centered between the two points
m = folium.Map(location=[(o_lat + d_lat) / 2, (o_lon + d_lon) / 2], zoom_start=11)

folium.Marker(
    [o_lat, o_lon],
    popup="Origin",
    icon=folium.Icon(color="red"),
).add_to(m)

folium.Marker(
    [d_lat, d_lon],
    popup="Destination",
    icon=folium.Icon(color="blue"),
).add_to(m)

m

## Comparison of Routes API results

In [ ]:
# routes duration time comparison (without caching)
## Nov 27

In [ ]:
# import data 300 od pairs with different departure times
data = pd.read_csv("../Data/routeresultadded_od300.csv")
origins = data[["oid", "oy", "ox"]]
destinations = data[["did", "dy", "dx"]]

# convert origins, destinations to geodataframe
origins = gpd.GeoDataFrame(
    origins,
    geometry=[Point(xy) for xy in zip(origins.ox, origins.oy, strict=False)],
)
destinations = gpd.GeoDataFrame(
    destinations,
    geometry=[Point(xy) for xy in zip(destinations.dx, destinations.dy, strict=False)],
)

# apply parsedataframetoLocations
originsLoc = parseDataframeToLocations(origins)
destinationsLoc = parseDataframeToLocations(destinations)

In [ ]:
data

In [ ]:
compare = []
for i, row in od_300.iterrows():
    x = calculateRoute(
        apikey=key,
        orig=originsLoc[i],
        dest=destinationsLoc[i],
        traffic="TRAFFIC_AWARE",
        departure_time=(2023, 11, 27, row["hour_of_day"], 0, 0),
    )
    compare.append(x)

In [ ]:
df_compare = createDataFrameFromResults(compare)
df_compare

In [ ]:
previous = data[["distance", "duration", "polyline"]]

In [ ]:
previous == df_compare

In [ ]:
dataframes = {
    "1126_distance": df_compare["distance"],
    "1126_duration": df_compare["duration"],
    "1119_distance": previous["distance"],
    "1119_duration": previous["duration"],
}

styles = {
    "1126_distance": "r-",
    "1126_duration": "r-",
    "1119_distance": "b-",
    "1119_duration": "b-",
}

In [ ]:
# Distance
plt.figure(figsize=(12, 6))
plt.plot(dataframes["1126_distance"], styles["1126_distance"], label="1126 Distance")
plt.plot(dataframes["1119_distance"], styles["1119_distance"], label="1119 Distance")
plt.title("Distance Comparison")
plt.ylabel("Distance")
plt.legend()
plt.show()

In [ ]:
# Duration
plt.figure(figsize=(12, 6))
plt.plot(dataframes["1126_duration"], styles["1126_duration"], label="1126 Duration")
plt.plot(dataframes["1119_duration"], styles["1119_duration"], label="1119 Duration")
plt.title("Duration Comparison")
plt.ylabel("Duration")
plt.legend()
plt.show()

In [ ]:
# calculate the difference (distance, duration)
difference = pd.DataFrame()
difference["distance"] = (
    (df_compare["distance"] - previous["distance"]) / previous["distance"]
) * 100
difference["duration"] = (
    (df_compare["duration"] - previous["duration"]) / previous["duration"]
) * 100

In [ ]:
difference

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

difference["distance"].plot(ax=axes[0])
axes[0].set_title("Distance")
axes[0].set_ylabel("Difference (%)")

difference["duration"].plot(ax=axes[1])
axes[1].set_title("Duration")
axes[1].set_ylabel("Difference (%)")

plt.tight_layout()
plt.show()